In [16]:
"""
CheXNet applied to brain tumor MRI images used for binary classification of tumor vs no tumor.
"""

import tensorflow.python.keras.backend as K
import numpy as np
import tensorflow as tf
from loss import WeightedCrossEntropyBinaryLoss
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Dense
import prepare_data
import glob
import os

In [17]:
# input_path = '../input/chest_xray/chest_xray/'
input_path = 'C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/chest_xray/'

# Checkpoint file path
checkpoint_filepath = 'C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/Checkpoint' 

# Hyperparameters
img_dims = 224
epochs = 10
batch_size = 16
val_batch_size = 64

# Getting the data
train_gen, test_gen, test_data, test_labels = prepare_data.prepare_data(img_dims, batch_size)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [18]:
class CheXNet(tf.keras.Model):
  """
  The model using modified DenseNet121.
  """

  def __init__ (self):
    """
    The constructor initiates the layers and their activation functions....
    """
    super().__init__()

    self.batch_size = 16
    self.input_size = 224
    self.output_size = 1 # since it's a binary classification task
    self.val_batch_size = 64  # This can be set any convenient value as per GPU capacity

    # Following will be set by get_data_stats() based on the dataset
    self.zero_weight = None
    self.one_weight = None
    self.train_steps = None
    self.val_steps = None

    # get_model() will initialize this to DenseNet121 model
    self.model = None
  
  def get_data_stats(self, train_data_path, val_data_path, class_map):
      """
      Computes normal Vs Pneumonia class distribution.
      :param train_data_path: path to training data. Samples os each class should be stored in separate folders
      :param val_data_path: path to validation data. Samples os each class should be stored in separate folders
      :param class_map: mapping of class index to folder names
      """

      # Count images in each class
      for _set in ['train', 'val', 'test']:
          n_normal = len(os.listdir(input_path + _set + '/NORMAL'))
          n_infect = len(os.listdir(input_path + _set + '/PNEUMONIA'))

      # compute class distribution
      self.w_class1 = float(n_normal)/(n_normal+n_infect)
      self.w_class0 = float(n_infect)/sum(n_normal+n_infect)

      # For convenience at train time, compute number of steps required to complete an epoch
      val_img_cnt = 0
      for key, value in class_map.items():
          imgs = glob.glob(val_data_path + value + "/*.jpg")
          val_img_cnt += len(imgs)

      self.train_steps = ((n_normal+n_infect) // self.batch_size) + 1
      self.val_steps = ((n_normal+n_infect) // self.val_batch_size) + 1  


  def get_model(self):
    
    # DenseNet121 expects number of channels to be 3
    input = Input(shape=(self.input_size, self.input_size, 3))

    # using pretrained DenseNet121 as the foundation of the model
    self.base_model = tf.keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet', input_tensor=None,
                                                                input_shape=None, pooling='avg', classes=2)
    self.output_layer = tf.keras.layers.Dense(self.output_size, activation=tf.nn.sigmoid)


    # Using weighted binary loss has been suggested in the paper
    loss = WeightedCrossEntropyBinaryLoss(self.zero_weight, self.one_weight)

    # Compile the model
    self.model=model.compile(optimizer=tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999),
                  loss=loss.weighted_binary_crossentropy, metrics=['accuracy'])
    
    return self.model

  @tf.function
  def call (self, input):
    """
    This function calls the model on new input and returns the output as tensors.

    Arguments:
    input -- denotes the input tensors
    """
    x = self.base_model(input)
    x = self.output_layer(x)
    return x

# Instantiate the CheXNet model
model = CheXNet()


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [19]:
#Fitting the model
history = model.fit(
    train_gen, epochs=10, validation_data=test_gen,
    callbacks=[reduce_lr, model_checkpoint])

# Load the model with the lowest validation loss
model.load_weights(checkpoint_filepath)

model.summary()


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.